In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
%matplotlib inline

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# import os
# os.chdir("/content/gdrive/My Drive/RandomForest")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
train_dataset = pd.read_csv('../../train_final.csv', names=["vx", "vy", "vz", "dx", "dy", "vfx", "vfy", "vfz", "afx", "afy", "afz", "num_v_labels", "ax", "ay", "az"])
test_dataset = pd.read_csv('../../test.csv', names=["vx", "vy", "vz", "dx", "dy", "vfx", "vfy", "vfz", "afx", "afy", "afz", "num_v_labels", "ax", "ay", "az"])
train_dataset = train_dataset.drop(train_dataset.index[0])
test_dataset = test_dataset.drop(test_dataset.index[0])
print("Shape of the training data is", train_dataset.shape)
print("Shape of the validation data is", test_dataset.shape)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape of the training data is (154371, 15)
Shape of the validation data is (29004, 15)


In [3]:
X_train = train_dataset.iloc[:, :12]
X_test = test_dataset.iloc[:, :12]
print("Shape of the X_train is", X_train.shape)
print("Shape of the X_test is", X_test.shape)

Shape of the X_train is (154371, 12)
Shape of the X_test is (29004, 12)


In [4]:
Y_train = train_dataset.iloc[:,12:]
Y_test = test_dataset.iloc[:, 12:]
print("Shape of the Y_train is", Y_train.shape)
print("Shape of the Y_test is", Y_test.shape)

Shape of the Y_train is (154371, 3)
Shape of the Y_test is (29004, 3)


In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import pickle

def grid_search(model_type, tuned_parameters, X_train, y_train, X_test, y_test, file_name):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        clf = GridSearchCV(model_type, tuned_parameters, cv=3)
        clf.fit(X_train, y_train)
        print("Best parameters set found on development set:")
        print("")
        print(clf.best_params_)
        print("")
        print("Saving the weights of the model")
        pickle.dump(clf, open(file_name, 'wb'))
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print("")
        y_true, y_pred = y_test, clf.predict(X_test)
        print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
        print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
        return y_pred, y_true

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

tuned_parameters = [{
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [10],
    'n_estimators': [100, 200]
}]

y_pred, y_true = grid_search(model, tuned_parameters, X_train, Y_train, X_test, Y_test, "random_forest_weights_lg.sav")